In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

from slim_gsgp.datasets.data_loader import load_pandas_df
import pandas as pd
import numpy as np
from slim_gsgp.main_gp import gp
from slim_gsgp.main_slim import slim
from slim_gsgp.main_gsgp import gsgp
from slim_gsgp.utils.utils import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import torch
from imblearn.over_sampling import SMOTENC, SMOTE

In [2]:
import slim_gsgp.evaluators.fitness_functions

In [3]:
print(slim_gsgp.evaluators.fitness_functions.train_sample_weights)
slim_gsgp.evaluators.fitness_functions.train_sample_weights = 100
print(slim_gsgp.evaluators.fitness_functions.train_sample_weights)

None
100


In [4]:
torch.round(torch.sigmoid(torch.tensor(0)))

tensor(0.)

In [5]:
(torch.tensor(0) > 0).float()

tensor(0.)

In [6]:
def print_scores(y_test, predictions):
    print("Roc Score:", roc_auc_score(y_test, predictions))
    print("F1 Score:", f1_score(y_test, predictions))
    print("Accuracy Score:", accuracy_score(y_test, predictions))

In [7]:
df = pd.read_csv('data/data_prepared/wilt.csv')

In [8]:
X, y = load_pandas_df(df, X_y=True)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.2)
#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.2)

In [10]:
y_train

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [11]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

def compute_class_weights_dict(y_true):
    """
    Compute class weights based on inverse frequency and return them as a normalized dictionary.
    The weights are normalized to sum to 1.

    Parameters
    ----------
    y_true : array-like
        Array of true class labels.

    Returns
    -------
    dict
        A dictionary where keys are class labels and values are the normalized class weights.
    """
    # Compute the class weights using 'balanced' strategy from sklearn
    class_weights = compute_class_weight('balanced', classes=np.unique(y_true), y=y_true)
    
    
    # Create a dictionary that maps class labels to their corresponding normalized weights
    class_weight_dict = {int(label): weight for label, weight in zip(np.unique(y_true), class_weights)}
    
    return class_weight_dict

class_weight_dict = compute_class_weights_dict(df['target'])
print(class_weight_dict)


{0: 0.5087038343919078, 1: 29.222972972972972}


In [12]:
import torch
import numpy as np

def compute_class_weights_torch(y):
    """
    y: list, numpy array, or 1D torch tensor of class labels
    returns: torch tensor of class weights
    """
    y = torch.tensor(y) if not torch.is_tensor(y) else y
    classes, counts = torch.unique(y, return_counts=True)
    total_samples = y.size(0)

    weights = total_samples / (len(classes) * counts.float())
    return weights


In [13]:
class_weights = compute_class_weights_torch(y_train)
class_weight_dict = {int(label): weight for label, weight in zip(np.unique(y_train), class_weights)}
class_weight_dict

{0: tensor(0.5091), 1: tensor(27.9032)}

In [14]:
class_weights

tensor([ 0.5091, 27.9032])

In [15]:
compute_class_weights_torch(y_train)

tensor([ 0.5091, 27.9032])

In [16]:
#class_weights = compute_class_weight('balanced', classes=np.unique(df['target']), y=df['target'])
train_sample_weights = torch.tensor([class_weights[int(label)] for label in y_train], dtype=torch.float32)
test_sample_weights = torch.tensor([class_weights[int(label)] for label in y_test], dtype=torch.float32)

In [20]:
train_sample_weights

tensor([0.5091, 0.5091, 0.5091,  ..., 0.5091, 0.5091, 0.5091])

In [18]:
compute_class_weight('balanced', classes=np.unique(np.array(y_train)), y=np.array(y_train))

array([ 0.50912301, 27.90322581])

In [19]:
slim_gsgp.evaluators.fitness_functions.train_sample_weights = train_sample_weights
slim_gsgp.evaluators.fitness_functions.test_sample_weights = test_sample_weights

In [35]:
slim_f1= slim(X_train=X_train, y_train=y_train, seed=1, log_path='log/slim_f1.csv', ms_lower = 0, ms_upper= 2, p_inflate = 0.9,
                  X_test=X_test, y_test=y_test, slim_version='SLIM+SIG1', pop_size=100, n_iter=1000, minimization=False, fitness_function='f1_score',
                  )
predictions = slim_f1.predict(X_test)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.08418658375740051 |   0.05797101557254791    |   0.08003664016723633  |      15          |
|     dataset_1           |       1      |   0.08418658375740051 |   0.05797101557254791    |   0.06297969818115234  |      15          |
|     dataset_1           |       2      |   0.08418658375740051 |   0.05797101557254791    |   0.0857551097869873   |      15          |
|     dataset_1           |       3      |   0.11666666716337204 |   0.06936416029930115    |   0.08393692970275879  |      40          |
|     dataset_1  

In [31]:
slim_gsgp.evaluators.fitness_functions.f1_score(y_test, predictions)

tensor(0.0971)

In [32]:
from experiments.utils import *
get_evaluation_dictionary(y_test, predictions)

{'rmse': 0.5553851127624512,
 'wrmse': 0.4424718916416168,
 'accuracy': 0.7849711179733276,
 'roc_auc': 0.8294255137443542,
 'f1_score': 0.09708737581968307,
 'precision': 0.05154639109969139,
 'recall': 0.8333333134651184}

In [30]:
predictions = torch.round(torch.sigmoid(predictions))
predictions

tensor([0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 

In [27]:
f1_score(y_test,predictions)

0.0970873786407767

In [24]:
f1_score(y_test,torch.round(torch.sigmoid(predictions)))

TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [99]:
y_test,torch.round(torch.sigmoid(predictions))

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]

In [ ]:
slim_f1.print_tree_representation()

('divide', ('add', ('subtract', ('multiply', 'x276', 'x44'), ('divide', 'constant_5.0', 'x398')), ('divide', ('divide', 'x251', 'x154'), ('divide', 'x86', 'x391'))), ('divide', ('subtract', ('divide', 'x366', 'x444'), ('divide', 'x290', 'x100')), ('add', ('add', 'constant_5.0', 'constant_5.0'), ('add', 'x211', 'x229')))) + f(('divide', 'x88', 'constant_2.0')) + f(('subtract', 'x247', 'x353')) + f(('add', 'x132', 'constant_3.0')) + f(('divide', 'x416', 'constant_3.0')) + f(('divide', 'x318', 'x29')) + f(('divide', 'x46', 'x121')) + f(('divide', 'x83', 'x216')) + f(('add', 'x299', 'constant__1.0'))


In [ ]:
gsgpp= gsgp(X_train=X_train, y_train=y_train, seed=1, log_path='log/slim_f1.csv', ms_lower = 0, ms_upper= 0.8, 
                  X_test=X_val, y_test=y_val,  pop_size=200, n_iter=1200, minimization=True, fitness_function='sigmoid_rmse',
                  )
predictions = slim_f1.predict(X_test)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_gsgp.py:145: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.4946924149990082  |   0.5845333337783813     |   0.3709878921508789   |      255         |
|     dataset_1           |       1      |   0.48815134167671204 |   0.4247274100780487     |   0.07489919662475586  |      37          |
|     dataset_1           |       2      |   0.47900307178497314 |   0.5256760716438293     |   0.08268499374389648  |      261         |
|     dataset_1           |       3      |   0.4705706238746643  |   0.4591137766838074     |   0.10103321075439453  |      49          |
|     dataset_1  

KeyboardInterrupt: 

In [ ]:
predictions

tensor([-1.4749,  2.9330, -1.9767,  0.0207, -0.8142,  2.5307, -1.9767, -2.8012,
         2.0821, -1.0505,  0.6891,  1.8713, -2.1033,  0.1766,  1.9324, -2.4011,
         1.7884,  2.7797, -2.1834,  1.3582, -1.3672,  1.7300, -1.9153,  3.7126,
        -0.8975, -2.5297, -1.1792,  0.9821, -2.5297, -1.6789, -0.3764,  2.0772,
        -0.0351, -0.8568, -1.0505, -1.3310,  1.1526,  1.5576, -0.4344,  4.6797,
         1.7620, -2.0893, -0.0898, -0.5644,  0.0834, -3.5989, -2.3974,  0.8215,
         1.4310, -2.8628, -0.4057,  2.3812,  2.5668,  0.2682, -2.1053, -2.5297,
        -1.9661,  0.4120,  1.3391, -2.8469,  0.0616,  0.0395, -2.1033,  1.1262,
        -1.9767,  2.7902, -2.8414,  2.4026, -0.2165, -1.2386,  0.5411,  0.5028,
         0.3812,  1.3036, -0.5854,  0.0694,  3.4145,  1.8789,  1.9163,  0.8508,
        -0.4155,  1.1225, -2.0894, -0.4164,  3.6867, -1.5949, -1.1545,  0.4291,
        -2.1033,  0.9535, -2.5297, -1.1772,  0.8511, -0.1400, -0.0192, -2.8628,
        -2.1033, -2.8414, -1.2546, -1.23

In [ ]:
torch.sigmoid()

In [ ]:
torch.sigmoid(predictions)

tensor([0.1862, 0.9495, 0.1217, 0.5052, 0.3070, 0.9263, 0.1217, 0.0573, 0.8891,
        0.2591, 0.6658, 0.8666, 0.1088, 0.5440, 0.8735, 0.0831, 0.8567, 0.9416,
        0.1013, 0.7955, 0.2031, 0.8494, 0.1284, 0.9762, 0.2896, 0.0738, 0.2352,
        0.7275, 0.0738, 0.1572, 0.4070, 0.8887, 0.4912, 0.2980, 0.2591, 0.2090,
        0.7600, 0.8260, 0.3931, 0.9908, 0.8535, 0.1101, 0.4776, 0.3625, 0.5208,
        0.0266, 0.0834, 0.6946, 0.8071, 0.0540, 0.3999, 0.9154, 0.9287, 0.5666,
        0.1086, 0.0738, 0.1228, 0.6016, 0.7923, 0.0548, 0.5154, 0.5099, 0.1088,
        0.7551, 0.1217, 0.9421, 0.0551, 0.9170, 0.4461, 0.2247, 0.6321, 0.6231,
        0.5942, 0.7864, 0.3577, 0.5173, 0.9682, 0.8675, 0.8717, 0.7007, 0.3976,
        0.7545, 0.1101, 0.3974, 0.9756, 0.1687, 0.2397, 0.6057, 0.1088, 0.7218,
        0.0738, 0.2356, 0.7008, 0.4650, 0.4952, 0.0540, 0.1088, 0.0551, 0.2219,
        0.2247, 0.8648, 0.9490, 0.1088, 0.5239, 0.1217, 0.4392, 0.5619, 0.0804,
        0.1086, 0.8666, 0.4061, 0.1217, 

In [ ]:
slim_f1.nodes_count

123

In [ ]:
X_train_over, y_train_over = SMOTE( random_state=42).fit_resample(X_train, y_train)
X_train_over = torch.tensor(X_train_over).float()
y_train_over = torch.tensor(y_train_over).float()

In [ ]:
slim_rmse = slim(X_train=X_train, y_train=y_train,
                  X_test=X_val, y_test=y_val, log_path='log/slim_rmse.csv', 
                   slim_version='SLIM+SIG2', pop_size=200, n_iter=1000, minimization=True, fitness_function='sigmoid_rmse',
                  ms_lower=0, ms_upper=0.5, p_inflate=0.5)

predictions = slim_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_slim.py:165: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.43729355931282043 |   0.4102255702018738     |   0.08507895469665527  |      9           |
|     dataset_1           |       1      |   0.43384045362472534 |   0.4082792103290558     |   0.10199856758117676  |      24          |
|     dataset_1           |       2      |   0.43384045362472534 |   0.4082792103290558     |   0.10250282287597656  |      24          |
|     dataset_1           |       3      |   0.4325994551181793  |   0.40630215406417847    |   0.10356354713439941  |      57          |
|     dataset_1  

In [ ]:
gsgp_f1 = gsgp(X_train=X_train, y_train=y_train,
                  X_test=X_val, y_test=y_val, reconstruct=True, log_path='log/gsgp_f1.csv', 
                  pop_size=500, n_iter=100, minimization=False, fitness_function='f1_score', seed = 0
                  )

predictions = gsgp_f1.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

NameError: name 'gsgp' is not defined

In [ ]:
gsgp_rmse = gsgp(X_train=X_train_over, y_train=y_train_over,
                  X_test=X_val, y_test=y_val, reconstruct=True, log_path='log/gsgp_rmse.csv',
                  pop_size=500, n_iter=500, minimization=True, fitness_function='sigmoid_rmse', seed = 0,
                  )

predictions = gsgp_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.478540301322937   |   0.5302056670188904     |   0.9723749160766602   |      5           |
|     dataset_1           |       1      |   0.4644612669944763  |   0.4699842631816864     |   0.625159740447998    |      11          |
|     dataset_1           |       2      |   0.46438002586364746 |   0.49265673756599426    |   0.6738924980163574   |      19          |
|     dataset_1           |       3      |   0.4607837200164795  |   0.4389142394065857     |   0.6179013252258301   |      23          |
|     dataset_1  

In [ ]:
gp_f1 = gp(X_train=X_train, y_train=y_train, p_xo=0.2,
                  X_test=X_val, y_test=y_val, pop_size=200, n_iter=100, minimization=False, fitness_function='f1_score', seed = 0,
                  )
predictions = gp_f1.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.44525253772735596 |   0.45308926701545715    |   0.18932867050170898  |      5           |
|     dataset_1           |       1      |   0.44525253772735596 |   0.45308926701545715    |   0.2543361186981201   |      5           |
|     dataset_1           |       2      |   0.44525253772735596 |   0.45308926701545715    |   0.25008082389831543  |      5           |
|     dataset_1           |       3      |   0.44525253772735596 |   0.45308926701545715    |   0.31145787239074707  |      5           |
|     dataset_1  

In [ ]:
y_test

tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.])

In [ ]:
gp_rmse = gp(X_train=X_train_over, y_train=y_train_over,
                  X_test=X_val, y_test=y_val, pop_size=200, n_iter=100, minimization=True, fitness_function='sigmoid_rmse', seed = 0,
                  )
predictions = gp_rmse.predict(X_test)
predictions = torch.round(torch.sigmoid(predictions))
print_scores(y_test, predictions)

c:\Users\leond\OneDrive\Desktop\MasterThesis\slim\slim_gsgp\main_gp.py:148: UserWarning: No dataset name set. Using default value of dataset_1.
  warnings.warn("No dataset name set. Using default value of dataset_1.")


Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     dataset_1           |       0      |   0.47669273614883423 |   0.5389356017112732     |   0.18155336380004883  |      5           |
|     dataset_1           |       1      |   0.47669273614883423 |   0.5389356017112732     |   0.1466360092163086   |      5           |
|     dataset_1           |       2      |   0.47613951563835144 |   0.5272106528282166     |   0.14223527908325195  |      5           |
|     dataset_1           |       3      |   0.4737030863761902  |   0.5282784700393677     |   0.1464700698852539   |      5           |
|     dataset_1  

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
rf = RandomForestClassifier(max_depth=2)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
print_scores(y_test, predictions)

gb = GradientBoostingClassifier(max_depth=2)
gb.fit(X_train, y_train)
predictions = gb.predict(X_test)
print_scores(y_test, predictions)

Roc Score: 0.5
F1 Score: 0.0
Accuracy Score: 0.7931034482758621
Roc Score: 0.5615942028985508
F1 Score: 0.25
Accuracy Score: 0.7931034482758621


In [ ]:
final_tree = gp(X_train=X_train, y_train=y_train, log_path='log/test.csv', max_depth=15,
                X_test=X_val, y_test=y_val, log_level=1, seed=2444,
                dataset_name='xyz', pop_size=200, n_iter=100)

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     xyz                 |       0      |   0.3630543351173401  |   0.3774508535861969     |   0.10421562194824219  |      5           |
|     xyz                 |       1      |   0.35974404215812683 |   0.3771507441997528     |   0.10279154777526855  |      15          |
|     xyz                 |       2      |   0.35974404215812683 |   0.3771507441997528     |   0.10576272010803223  |      15          |
|     xyz                 |       3      |   0.35974404215812683 |   0.3771507441997528     |   0.1434328556060791   |      15          |
|     xyz        